# Matter

Matter evolution equations follow from stress energy conservation,
$$
\nabla_a T^{ab} = 0,
$$
combined with any additional constituitive equations (continuity, Maxwell equations, and so on). By contracting with a tetrad vector $e^{(j)}_b$ and using that $\nabla_a V^a = (-g)^{-1/2} \partial_a ((-g)^{1/2} V^a)$, we can write stress energy conservation as
$$
\partial_a \left( \sqrt{-g} e^{(j)}_b T^{ab} \right) = \sqrt{-g} T^{ab} \nabla_a e^{(j)}_b.
$$
This is a set of *hyperbolic conservation laws*
$$
\partial_t \mathbf{q} + \partial_j \mathbf{f}^{(j)}(\mathbf{q}) = \mathbf{s},
$$
where the *conserved* quantities $\mathbf{q}$ are **roughly** the number density, momentum density, and energy density, the *fluxes* $\mathbf{f}^{(j)}$ are the corresponding currents, and the *sources* $\mathbf{s}$ are the right-hand side of the above equation.

The two prototypical examples of conservation laws are the *advection equation*
$$
\partial_t q + \partial_x (v q) = 0
$$
where the advection velocity $v$ is constant, and *Burgers equation*
$$
\partial_t q + \partial_x \left( \frac{1}{2} q^2 \right) = 0.
$$

The *local* behaviour of the solutions can be found by looking at *characteristics*: curves $x(t)$ along which the solution is constant. For the advection equation, characteristics obey $x / t = v$ and so are straight lines. For Burgers equation, characteristics obey $x / t = q$ and so *depend on the data itself*. This means that initial data can see steep gradients smooth out, or steepen until the solution becomes discontinuous. The latter behaviour is called *shock formation*.

## Finite differencing

As with the wave equation we will use central differencing and RK2, with a sine wave initial profile.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipympl

In [ ]:
def grid(Npoints, xl=-1.0, xr=1.0):
    """
    Npoints is the number of interior points
    """
    
    dx = (xr - xl) / Npoints
    return dx, np.linspace(xl-dx/2.0, xr+dx/2.0, Npoints+2)

def apply_boundaries(q):
    """
    Periodic boundaries
    """
    N = q.shape[1] - 2

    q[:, 0] = q[:, N]
    q[:, N+1] = q[:, 1]
    
    return q

def RK2_step(q, RHS, apply_boundaries, dt, dx):
    """
    RK2 method
    """

    rhs = RHS(q, dx)
    qp = q + dt * rhs
    qp = apply_boundaries(qp)
    rhs_p = RHS(qp, dx)
    qnew = 0.5 * (q + qp + dt * rhs_p)
    qnew = apply_boundaries(qnew)

    return qnew

In [ ]:
def flux_advection(q):
    v = 1.0
    return v*q

def flux_burgers(q):
    return 0.5*q**2

def RHS_advection(q, dx):
    dqdt = np.zeros_like(q)
    dqdt[0, 1:-1] = -1.0 / (2.0*dx)*(flux_advection(q[0, 2:]) - flux_advection(q[0, :-2]))
    return dqdt

def RHS_burgers(q, dx):
    dqdt = np.zeros_like(q)
    dqdt[0, 1:-1] = -1.0 / (2.0*dx)*(flux_burgers(q[0, 2:]) - flux_burgers(q[0, :-2]))
    return dqdt

def initial_data(x):
    q = np.zeros((1, x.shape[0]))
    q[0, :] = np.sin(np.pi*(x-0.4))
    return q

In [ ]:
Npoints = 50
dx, x = grid(Npoints)
dt = dx / 4
q0 = apply_boundaries(initial_data(x))
q_advection = apply_boundaries(initial_data(x))
q_burgers = apply_boundaries(initial_data(x))
Nsteps = int(0.3 / dt)
for n in range(Nsteps):
    q_advection = RK2_step(q_advection, RHS_advection, apply_boundaries, dt, dx)
    q_burgers = RK2_step(q_burgers, RHS_burgers, apply_boundaries, dt, dx)

plt.figure()
plt.plot(x, q0[0, :], 'b--', label="Initial data")
plt.plot(x, q_advection[0, :], 'k-', label=r"Advection")
plt.plot(x, q_burgers[0, :], 'r-', label=r"Burgers")
plt.title("Time 0.3")
plt.xlabel(r"$x$")
plt.ylabel(r"$q$")
plt.xlim(-1, 1)
plt.legend()
plt.show()

In [ ]:
Npoints = 50
dx, x = grid(Npoints)
dt = dx / 4
q0 = apply_boundaries(initial_data(x))
q_advection = apply_boundaries(initial_data(x))
q_burgers = apply_boundaries(initial_data(x))
Nsteps = int(0.6 / dt)
for n in range(Nsteps):
    q_advection = RK2_step(q_advection, RHS_advection, apply_boundaries, dt, dx)
    q_burgers = RK2_step(q_burgers, RHS_burgers, apply_boundaries, dt, dx)

plt.figure()
plt.plot(x, q0[0, :], 'b--', label="Initial data")
plt.plot(x, q_advection[0, :], 'k-', label=r"Advection")
plt.plot(x, q_burgers[0, :], 'r-', label=r"Burgers")
plt.title("Time 0.6")
plt.xlabel(r"$x$")
plt.ylabel(r"$q$")
plt.xlim(-1, 1)
plt.legend()
plt.show()

We see the expected behaviour *until* the shock forms. The advection solution is the initial data shifted along by the advection velocity. The solution to Burgers equation steepens until the shock forms, at which point the numerical method breaks down. Oscillations appear in the numerical solution.

We need to check what happens as resolution is increased.

In [ ]:
plt.figure()
for Npoints in [100, 200, 400]:
    dx, x = grid(Npoints)
    dt = dx / 4
    q0 = apply_boundaries(initial_data(x))
    q_burgers = apply_boundaries(initial_data(x))
    Nsteps = int(0.6 / dt)
    for n in range(Nsteps):
        q_burgers = RK2_step(q_burgers, RHS_burgers, apply_boundaries, dt, dx)

    plt.plot(x, q_burgers[0, :], label=rf"{Npoints} points")
plt.title("Time 0.6")
plt.xlabel(r"$x$")
plt.ylabel(r"$q$")
plt.xlim(-1, 1)
plt.legend()
plt.show()

The magnitude of the oscillations is not converging with resolution, but the frequency is. This indicates a problem with the numerical method, and often these oscillations will blow up with time.

## Finite volumes

Looking at the original conservation law
$$
\partial_t q + \partial_x f(q) = 0
$$
we see that it *makes no sense* when $q$ is discontinuous. This *strong form* of the PDE cannot describe shocks. Instead, we consider the spacetime volume $x \in [x_{i-1/2}, x_{i+1/2}]$ and $t \in [t^{n}, t^{n+1}]$ and integrate the conservation law over this volume. This gives
$$
\int_{x_{i-1/2}}^{x_{i+1/2}} q(x, t^{n+1}) - q(x, t^{n}) + \int_{t^n}^{t^{n+1}} f(q(x_{i+1/2}, t)) - f(q(x_{i-1/2}, t)) = 0.
$$
By *defining* appropriate integral average variables we can write this as
$$
\hat{q}_i^{n+1} - \hat{q}_i^n + \frac{\Delta t}{\Delta x} \left( \hat{f}_{i+1/2}^{n} - \hat{f}_{i-1/2}^{n} \right) = 0.
$$
In particular, $\hat{q}$ is spatial average of $q$ over the volume, and $\hat{f}$ is the time integral of the flux through the spatial boundary of the volume.

The crucial point of this *weak form* of the PDE is that there are no derivatives, so  it can handle shocks. It is also immediately set up for a numerical method: we just need to specify how to compute $\hat{f}(q)$ from the integral averages $\hat{q}$.

### Godunov methods

The simplest choice is to approximate $q$ as being constant over each cell volume. This means that $\hat{f}_{i+1/2}$ depends on $\hat{q}_i$ and $\hat{q}_{i+1}$, which are constant values. In the case of the advection equation, this can be solved exactly, giving
$$
\hat{f}_{i+1/2} = \begin{cases} f(\hat{q}_i) & \text{if } v > 0, \\ f(\hat{q}_{i+1}) & \text{if } v < 0. \end{cases}
$$
In nonlinear cases such as Burger's equation the exact is more complex, so we will use the *approximate* solution given by the *Lax-Friedrichs* flux
$$
\hat{f}_{i+1/2} = \frac{1}{2} \left( f(\hat{q}_i) + f(\hat{q}_{i+1}) \right) - \lambda \left( \hat{q}_{i+1} - \hat{q}_i \right).
$$
Here $\lambda$ has to be chosen to be larger than the maximum wave speed in the problem.

### Implement and check

Apply Godunov to the Burger's equation and check how the shock development behaves with increasing resolution.

In [ ]:
def lf_flux(qL, qR, lamda):
    return 0.5*(flux_burgers(qL) + flux_burgers(qR) - lamda * (qR - qL))

def godunov_burgers_step(q, apply_boundaries, dt, dx):
    lamda = np.max(q)
    qnew = q.copy()
    for i in range(1, q.shape[1]-1):
        qnew[:, i] = q[:, i] - dt / dx * (lf_flux(q[:, i], q[:, i+1], lamda) - lf_flux(q[:, i-1], q[:, i], lamda))
    qnew = apply_boundaries(qnew)
    return qnew

In [ ]:
plt.figure()
for Npoints in [100, 200, 400]:
    dx, x = grid(Npoints)
    dt = dx / 4
    q0 = apply_boundaries(initial_data(x))
    q_burgers = apply_boundaries(initial_data(x))
    Nsteps = int(0.6 / dt)
    for n in range(Nsteps):
        q_burgers = godunov_burgers_step(q_burgers, apply_boundaries, dt, dx)

    plt.plot(x, q_burgers[0, :], label=rf"{Npoints} points")
plt.title("Time 0.6")
plt.xlabel(r"$x$")
plt.ylabel(r"$q$")
plt.xlim(-1, 1)
plt.legend()
plt.show()

We see that the shock is captured without oscillations, and that the solution converges with resolution.

# Relativistic hydrodynamics

Working with a perfect fluid, the stress energy tensor is
$$
T^{ab} = \rho h u^a u^b + p g^{ab},
$$
where $\rho$ is the specific rest mass density, $h = 1 + \epsilon + p / \rho$ is the specific enthalpy, $\epsilon$ is the specific internal energy, $u^a$ is the fluid four-velocity, and $p$ is the pressure. The conservation laws follow from the continuity equation ($\nabla_a (\rho u^a) = 0$) and stress-energy conservation as above, and can be written (in $1+1$ Minkowski spacetime, using Cartesian coordinates) as
$$
\partial_t \mathbf{q} + \partial_x \mathbf{f}(\mathbf{q}) = \mathbf{0},
$$
where
$$
\mathbf{q} = \begin{pmatrix} D \\ S \\ \tau \end{pmatrix} = \begin{pmatrix} \rho W \\ \rho h W^2 v \\ \rho h W^2 - p - \rho W \end{pmatrix},
$$
with $W = u^t$ the Lorentz factor, and $v = u^x / u^t$ the fluid 3-velocity, linked by $W = (1 - v^2)^{-1/2}$. The fluxes are
$$
\mathbf{f}(\mathbf{q}) = \begin{pmatrix} D v \\ S v + p \\ (\tau + p) v \end{pmatrix}.
$$
The equations are closed by an equation of state that we typically write in the form $p = p(\rho, \epsilon)$ or $p = p(\rho, h)$.

Before we can apply Godunov-type methods to this system we need three additional pieces of information.

First, we need the maximum signal speed $\lambda$ to complete our Lax-Friedrichs flux. In principle this is the relativistic sound speed. However, this is bounded by the speed of light, so for simplicity we can just set $\lambda = 1$.

Second, we need an equation of state to close the system. Again for simplicity, we will use the ideal gas equation of state $p = (\Gamma - 1) \rho \epsilon$, where $\Gamma$ is the adiabatic index.

Finally, we need to compute the primitive variables $\rho$, $v$, and $p$ from the conserved variables $\mathbf{q}$. This is the *conservative to primitive* step, which is annoyingly costly in general. Here we will use a reasonably general approach. First guess the pressure, and denote the guess $\bar{p}$. Then given the conserved variables, compute
$$
\begin{aligned}
&& \frac{S^2}{(\tau + \bar{p} + D)^2} &= v^2 \\
\implies && W &= (1 - v^2)^{-1/2} \\
\implies && \rho &= D / W \\
\implies && h &= \frac{\tau + \bar{p} + D}{\rho W^2} \\
\implies && \epsilon &= h - 1 - \bar{p} / \rho \, .
\end{aligned}
$$
Now we can compute the pressure from the equation of state. This pressure can be compared to the guess. We can then use any root-finding algorithm to find the correct pressure (see the use of the bisection method in the horizon-finding section).

In [ ]:
from scipy.optimize import root_scalar

def p_eos(rho, epsilon, gamma=1.4):
    return (gamma - 1) * rho * epsilon

def h_eos(rho, p, gamma=1.4):
    return 1 + gamma * p / (rho * (gamma - 1))

def c2p_root(pbar, q):
    D, S, tau = q
    v2 = S**2 / (tau + pbar + D)**2
    W = 1.0 / np.sqrt(1.0 - v2)
    rho = D / W
    h = (tau + pbar + D) / (rho * W**2)
    epsilon = h - 1.0 - pbar / rho
    return pbar - p_eos(rho, epsilon)

def c2p(q):
    D, S, tau = q
    res = root_scalar(c2p_root, args=(q), bracket=[1.0e-10, 1.0e6])
    p = res.root
    v2 = S**2 / (tau + p + D)**2
    W = 1.0 / np.sqrt(1.0 - v2)
    rho = D / W
    h = h_eos(rho, p)
    v = S / (rho * h * W**2)
    return rho, v, p

In [ ]:
def flux_srhd(q, gamma=1.4):
    D, S, tau = q
    rho, v, p = c2p(q)
    h = h_eos(rho, p)
    flux = np.zeros_like(q)
    flux[0] = D * v
    flux[1] = S * v + p
    flux[2] = (tau + p) * v
    return flux

def lf_flux_srhd(qL, qR):
    return 0.5*(flux_srhd(qL) + flux_srhd(qR) -  (qR - qL))

def godunov_srhd_step(q, apply_boundaries, dt, dx):
    qnew = q.copy()
    for i in range(1, q.shape[1]-1):
        qnew[:, i] = q[:, i] - dt / dx * (lf_flux_srhd(q[:, i], q[:, i+1]) - lf_flux_srhd(q[:, i-1], q[:, i]))
    qnew = apply_boundaries(qnew)
    return qnew

In [ ]:
def initial_data(x):
    rhoL = 1.0
    rhoR = 0.125
    pL = 1.0
    pR = 0.1
    hL = h_eos(rhoL, pL)
    hR = h_eos(rhoR, pR)
    DL = rhoL
    DR = rhoR
    SL = 0
    SR = 0
    tauL = rhoL*hL - pL - DL
    tauR = rhoR*hR - pR - DR
    q = np.zeros((3, len(x)))
    for i, xx in enumerate(x):
        if xx < 0:
            q[:, i] = DL, SL, tauL
        else:
            q[:, i] = DR, SR, tauR
    return q

def apply_boundaries(q):
    q[:, 0] = q[:, 1]
    q[:, -1] = q[:, -2]
    return q

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 12), sharex=True)
names = [r"$D$", r"$S$", r"$\tau$"]
for Npoints in [100, 200, 400]:
    dx, x = grid(Npoints)
    dt = dx / 4
    q0 = apply_boundaries(initial_data(x))
    q_srhd = apply_boundaries(initial_data(x))
    Nsteps = int(0.8 / dt)
    for n in range(Nsteps):
        q_srhd = godunov_srhd_step(q_srhd, apply_boundaries, dt, dx)

    for i, ax in enumerate(axes):
        ax.plot(x, q_srhd[i, :], label=rf"{Npoints} points")
for i, ax in enumerate(axes):
    ax.set_ylabel(names[i])
    ax.set_xlim(-1, 1)
    ax.legend()
plt.suptitle("Time 0.8")
axes[-1].set_xlabel(r"$x$")
plt.show()

We can see the expected structure of the shock tube: a rarefaction wave on the left, a linear contact discontinuity in the centre (propagating slowly to the right at $x \sim 0.3$) and a shock wave on the right (at $x \sim 0.6$).

However, all of the waves are smeared out and the improvement with resolution is slow. This is because Godunov's method is only first order accurate. Higher accuracy (*HRSC*) methods are possible, but noteably more complex when discontinuities arise.

## Conditioning

One key question is the accuracy of the steps taken. Usually we expect any one single step in the calculation to be inaccurate either at the level of floating point accuracy (numbers can only be stored with a certain precision, typically 16 significant figures) or truncation accuracy (for example, approximations to derivatives have larger errors, depending on the grid size). We then typically expect errors to roughly add up in magnitude, slowly degrading the accuracy of the final result.

However, sometimes things are *much worse* than this. Individual operations can hugely magnify errors, rather than just making them add up.

Consider the linear system
$$
\begin{pmatrix} \varepsilon & 1 \\ 1 & 1 \end{pmatrix} \mathbf{x} = \begin{pmatrix} 1 + \varepsilon \\ 2 \end{pmatrix} \, .
$$
This clearly has the solution $\mathbf{x} = (1, 1)^T$. However, let us solve it *assuming* that $\varepsilon$ is small enough so that, in floating point precision, $1 + \varepsilon \to 1$. Using Gaussian Elimination we subtract $\varepsilon^{-1}$ times the first row of the system from the second, and rescale the first row as well, giving
$$
\begin{pmatrix} 1 & \varepsilon^{-1} \\ 0 & - \varepsilon^{-1} \end{pmatrix} \mathbf{x} = \begin{pmatrix} \varepsilon^{-1} \\ - \varepsilon^{-1} \end{pmatrix} \, .
$$
Solving via back-substitution gives $x_2 = 1$ and $x_1 + \varepsilon^{-1} x_2 = \varepsilon^{-1}$, for a solution of $\mathbf{x} = (0, 1)^T$.

In this case, a poor choice of numerical algorithm means that perturbing the coefficients of the matrix by $\varepsilon \ll 1$ leads to a change in the solution of $\mathcal{O}(1)$. These types of algorithms are called *poorly conditioned*. We quantify this using a *condition number* $\kappa$. For a given operation, $\kappa$ should bound the amount that uncertainty or inaccuracy in the input is *magnified* by a given operation. A condition number $\kappa \sim 10$ is fine; a condition number of $\kappa \sim 10^{16}$ means that the limitations of floating point arithmetic alone will be magnified by this one operation so that even the first significant figure might be wrong.

The particular problem for relativistic hydrodynamics is that there are regimes where the conservative to primitive algorithm is poorly conditioned. This is particularly an issue at low temperatures.

In [ ]:
rho = 1
v = 0.1
W = 1 / np.sqrt(1 - v**2)
D = rho * W
epsilons = np.logspace(-6, 1)
d_eps_in = 1e-8
kappa_tau_eps = np.zeros_like(epsilons)
for i, epsilon in enumerate(epsilons):
    p = p_eos(rho, epsilon)
    h = 1 + epsilon + p / rho
    tau = rho * h * W**2 - p - D
    p_new = p_eos(rho, epsilon+d_eps_in)
    h_new = 1 + epsilon+d_eps_in + p_new / rho
    S_new = rho * h_new * W**2 * v
    tau_new = rho * h_new * W**2 - p_new - D
    d_tau = abs(tau_new - tau)
    rho_new, v_new, p_new = c2p(np.array([D, S_new, tau_new]))
    epsilon_new = h_eos(rho_new, p_new) - p_new / rho_new - 1
    d_eps_out = abs(epsilon_new - epsilon)
    kappa_tau_eps[i] = tau / epsilon * d_eps_out / d_tau

plt.figure()
plt.loglog(epsilons, kappa_tau_eps)
plt.xlabel(r"$\epsilon$")
plt.ylabel(r"$\kappa_{\tau \to \epsilon}$")
plt.show()

The values of the condition number here are not catastrophic, but for "real" equations of state the values are much worse, and the trend the same.